In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df=pd.read_csv('Weather Data.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date/Time         8784 non-null   object 
 1   Temp_C            8784 non-null   float64
 2   Dew Point Temp_C  8784 non-null   float64
 3   Rel Hum_%         8784 non-null   int64  
 4   Wind Speed_km/h   8784 non-null   int64  
 5   Visibility_km     8784 non-null   float64
 6   Press_kPa         8784 non-null   float64
 7   Weather           8784 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 549.1+ KB


In [5]:
df.duplicated().sum()

np.int64(0)

In [6]:
df.isna().sum()

,0
Date/Time,0
Temp_C,0
Dew Point Temp_C,0
Rel Hum_%,0
Wind Speed_km/h,0
Visibility_km,0
Press_kPa,0
Weather,0


In [7]:
df.head()

,Date/Time,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,1/1/2012 0:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1/1/2012 1:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,1/1/2012 2:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,1/1/2012 3:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,1/1/2012 4:00,-1.5,-3.3,88,7,4.8,101.23,Fog


In [8]:
df.columns

Index(['Date/Time', 'Temp_C', 'Dew Point Temp_C', 'Rel Hum_%',
       'Wind Speed_km/h', 'Visibility_km', 'Press_kPa', 'Weather'],
      dtype='object')

In [9]:
df['Date/Time']

,Date/Time
0,1/1/2012 0:00
1,1/1/2012 1:00
2,1/1/2012 2:00
3,1/1/2012 3:00
4,1/1/2012 4:00
...,...
8779,12/31/2012 19:00
8780,12/31/2012 20:00
8781,12/31/2012 21:00
8782,12/31/2012 22:00


In [10]:
df.Weather.value_counts()

,count
Weather,
Mainly Clear,2106
Mostly Cloudy,2069
Cloudy,1728
Clear,1326
Snow,390
Rain,306
Rain Showers,188
Fog,150
"Rain,Fog",116


In [11]:
df.Weather=df.Weather.apply(lambda x:'Cloudy' if 'Cloudy' in x else x)


In [12]:
df.Weather=df.Weather.apply(lambda x:'Clear' if 'Clear' in x else x)


In [13]:
df.Weather=df.Weather.apply(lambda x:'Snow' if 'Snow' in x else x)


In [14]:
df.Weather=df.Weather.apply(lambda x:'Rain' if 'Rain' in x else x)


In [15]:
df.Weather=df.Weather.apply(lambda x:'Fog' if 'Fog' in x else x)


In [16]:
df.Weather=df.Weather.apply(lambda x:'other' if x in ['Drizzle','Haze','Freezing Drizzle','Freezing Drizzle,Haze','Thunderstorms'] else x)


In [17]:
df.head()

,Date/Time,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,1/1/2012 0:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1/1/2012 1:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,1/1/2012 2:00,-1.8,-3.4,89,7,4.0,101.26,Fog
3,1/1/2012 3:00,-1.5,-3.2,88,6,4.0,101.27,Fog
4,1/1/2012 4:00,-1.5,-3.3,88,7,4.8,101.23,Fog


In [18]:
df.sample(10)

,Date/Time,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
2106,3/28/2012 18:00,1.3,-0.2,90,13,16.1,99.63,other
2920,5/1/2012 16:00,9.9,7.9,87,9,4.8,101.17,Fog
462,1/20/2012 6:00,-7.6,-14.3,59,28,25.0,101.13,Cloudy
8153,12/5/2012 17:00,-0.7,-8.3,56,30,25.0,101.66,Cloudy
6481,9/27/2012 1:00,8.3,6.3,87,9,25.0,101.54,Clear
8316,12/12/2012 12:00,-2.7,-6.5,75,7,19.3,102.14,Cloudy
5187,8/4/2012 3:00,21.8,14.8,64,6,25.0,101.61,Cloudy
8015,11/29/2012 23:00,-9.4,-15.3,62,20,25.0,102.42,Clear
2836,4/28/2012 4:00,-1.3,-9.8,52,24,25.0,101.60,Cloudy
8183,12/6/2012 23:00,-1.1,-6.0,69,7,25.0,101.73,Cloudy


In [19]:
df.Weather.value_counts()

,count
Weather,
Cloudy,3797
Clear,3432
Rain,662
Snow,583
Fog,241
other,69


In [20]:
df.Weather=df['Weather'].apply(
    lambda x: 0 if x == 'Cloudy'
    else 1 if x == 'Clear'
    else 3 if x == 'Rain'
    else 4 if x == 'Snow'
    else 5 if x == 'fog'
    else 6
)


In [21]:
df.Weather.value_counts()

,count
Weather,
0,3797
1,3432
3,662
4,583
6,310


In [22]:
df.head()

,Date/Time,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,1/1/2012 0:00,-1.8,-3.9,86,4,8.0,101.24,6
1,1/1/2012 1:00,-1.8,-3.7,87,4,8.0,101.24,6
2,1/1/2012 2:00,-1.8,-3.4,89,7,4.0,101.26,6
3,1/1/2012 3:00,-1.5,-3.2,88,6,4.0,101.27,6
4,1/1/2012 4:00,-1.5,-3.3,88,7,4.8,101.23,6


In [23]:
df.drop(['Date/Time'],axis=1, inplace=True)

In [24]:
df.head(1)

,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,-1.8,-3.9,86,4,8.0,101.24,6


In [25]:
X=df.drop(['Weather'],axis=1)

In [26]:
y=df.Weather

In [27]:
X_train,X_test,y_train,y_test = train_test_split (X,y,test_size=0.2, random_state=42)

In [28]:
from sklearn.linear_model import LinearRegression

In [29]:
lr=LinearRegression()

In [30]:
lr.fit(X_train,y_train)

LinearRegression()

In [31]:
y_pred_train=lr.predict(X_train)

In [32]:
y_pred_test=lr.predict(X_test)

In [33]:
from sklearn.metrics import r2_score, mean_absolute_error

In [34]:
r2_score(y_pred_train,y_train),r2_score(y_pred_test,y_test)

(-1.0571925769640629, -1.204186865235735)

In [35]:
dct=DecisionTreeClassifier()

In [36]:
dct.fit(X_train,y_train)

DecisionTreeClassifier()

In [37]:
y_pred_train=dct.predict(X_train)

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
accuracy_score(y_pred_train,y_train)

1.0

In [40]:
y_pred_test=dct.predict(X_test)

In [41]:
accuracy_score(y_test,y_pred_test)


0.6340352874217416

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
Lr=LogisticRegression()

In [44]:
Lr.fit(X_train,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [45]:
y_pred_train=Lr.predict(X_train)

In [46]:
y_pred_test=Lr.predict(X_test)

In [47]:
r2_score(y_test,y_pred_test)

0.32175471765419317

In [48]:
r2_score(y_train,y_pred_train)

0.3544394895286421

In [49]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,classification_report

In [50]:
print(classification_report(y_train,y_pred_train))

              precision    recall  f1-score   support

           0       0.55      0.60      0.57      3025
           1       0.59      0.57      0.58      2767
           3       0.56      0.33      0.41       535
           4       0.71      0.66      0.69       454
           6       0.58      0.56      0.57       246

    accuracy                           0.57      7027
   macro avg       0.60      0.55      0.56      7027
weighted avg       0.58      0.57      0.57      7027



In [51]:
print(classification_report(y_test,y_pred_test))

              precision    recall  f1-score   support

           0       0.56      0.60      0.58       772
           1       0.57      0.59      0.58       665
           3       0.55      0.33      0.41       127
           4       0.68      0.63      0.65       129
           6       0.60      0.48      0.53        64

    accuracy                           0.57      1757
   macro avg       0.59      0.53      0.55      1757
weighted avg       0.57      0.57      0.57      1757



In [52]:
confusion_matrix(y_train,y_pred_train)

array([[1829, 1072,   80,   36,    8],
       [1156, 1589,   12,    8,    2],
       [ 247,   28,  174,   32,   54],
       [  94,   18,    5,  301,   36],
       [  18,    2,   41,   47,  138]])

In [53]:
confusion_matrix(y_pred_test,y_test)

array([[460, 267,  57,  37,   5],
       [281, 391,   6,   4,   0],
       [ 17,   1,  42,   1,  16],
       [ 12,   4,  11,  81,  12],
       [  2,   2,  11,   6,  31]])

In [54]:
trf2=ColumnTransformer(['imputed_Columns',SimpleImputer(),[0,6]], remainder='passthrough')

In [55]:
trf2 = ColumnTransformer([
    ('imputed_Columns', SimpleImputer(), [0, 5])
], remainder='passthrough')


In [56]:
trf3 = ColumnTransformer([('scale',MinMaxScaler(),slice(0,10))])

In [57]:
trf4 = DecisionTreeClassifier()

In [58]:
pipe=make_pipeline(trf2,trf3,trf4)

In [59]:
pipe = Pipeline([
    ('imputer', trf2),
    ('scaler', trf3),
    ('model', DecisionTreeClassifier())
])

In [60]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputed_Columns',
                                                  SimpleImputer(), [0, 5])])),
                ('scaler',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('model', DecisionTreeClassifier())])

In [61]:
y_pred_train=pipe.predict(X_train)

In [62]:
y_pred_test=pipe.predict(X_test)

In [63]:
from sklearn.metrics import accuracy_score

In [64]:
accuracy_score(y_test,y_pred_test)

0.6289129197495731

In [65]:
accuracy_score(y_pred_train,y_train)

1.0

In [66]:
df=pd.read_csv('Weather Data.csv')

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date/Time         8784 non-null   object 
 1   Temp_C            8784 non-null   float64
 2   Dew Point Temp_C  8784 non-null   float64
 3   Rel Hum_%         8784 non-null   int64  
 4   Wind Speed_km/h   8784 non-null   int64  
 5   Visibility_km     8784 non-null   float64
 6   Press_kPa         8784 non-null   float64
 7   Weather           8784 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 549.1+ KB


In [68]:
df.isna().sum()

,0
Date/Time,0
Temp_C,0
Dew Point Temp_C,0
Rel Hum_%,0
Wind Speed_km/h,0
Visibility_km,0
Press_kPa,0
Weather,0


In [69]:
df.duplicated().sum()

np.int64(0)

In [70]:
df.head()

,Date/Time,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,1/1/2012 0:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1/1/2012 1:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,1/1/2012 2:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,1/1/2012 3:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,1/1/2012 4:00,-1.5,-3.3,88,7,4.8,101.23,Fog


In [71]:
df.Weather.value_counts()

,count
Weather,
Mainly Clear,2106
Mostly Cloudy,2069
Cloudy,1728
Clear,1326
Snow,390
Rain,306
Rain Showers,188
Fog,150
"Rain,Fog",116


In [72]:
df.Weather=df.Weather.apply(lambda x:'Cloudy' if 'Cloudy' in x else x)

In [73]:
df.Weather=df.Weather.apply(lambda x:'Clear' if 'Clear' in x else x)

In [74]:
df.Weather=df.Weather.apply(lambda x:'Rain' if 'Rain' in x else x)

In [75]:
df.Weather=df.Weather.apply(lambda x:'Fog' if 'Fog' in x else x)

In [76]:
df.Weather=df.Weather.apply(lambda x:'Snow' if 'Snow' in x else x)

In [77]:
df.Weather.value_counts()

,count
Weather,
Cloudy,3797
Clear,3432
Rain,689
Snow,500
Fog,297
Drizzle,41
Haze,16
Freezing Drizzle,7
"Freezing Drizzle,Haze",3


In [78]:
df.Weather=df.Weather.apply(lambda x:'others' if x in ['Drizzle','Haze','Freezing Drizzle','Freezing Drizzle,Haze','Thunderstorms'] else x)